In [ ]:
import pandas as pd
import csv
import os,json
import networkx as nx
import matplotlib.pyplot as plt
#os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.listdir("/content/gdrive/My Drive/NPPI")
path="/content/gdrive/My Drive/NPPI"

In [ ]:
#nx.write_gexf(G, "test.gexf") #READ AND WRITE LOADED GRAPH 
#G=nx.read_gexf("test.gexf")

In [ ]:
G1=nx.Graph() #Graph initialized to plot all the nodes and edges of NIP RW dataset

In [ ]:
NIP_Proteins = set()

In [ ]:
with open(path+'/BMCData/H. sapiens/NIP-RW.txt','r') as file:
  for i,line in enumerate(file):
          ln=line.split("\n")[0].split()
          G1.add_edge(ln[0],ln[1])

          # reading each protein        
          for p in ln:
    
              # displaying the protein          
              #print(p) 
              NIP_Proteins.add(p.strip())

print (i)
print (len(G1.nodes), len(G1.edges))

print (len(NIP_Proteins))

6709
2384 6710
2384


In [ ]:
G2=nx.Graph() #First half of the NIP RW protein pairs in G2 graph
G3=nx.Graph() #Second half of the NIP RW protein pairs in G3 graph (-ve)

In [ ]:
with open(path+'/BMCData/H. sapiens/NIP-RW.txt','r') as file:
  for i,line in enumerate(file):
          if (i < 3355):
            ln=line.split("\n")[0].split()
            G2.add_edge(ln[0],ln[1])

          else: 
            ln=line.split("\n")[0].split()
            G3.add_edge(ln[0],ln[1])

print (i)
print (len(G2.nodes), len(G2.edges))
print (len(G3.nodes), len(G3.edges))

6709
2384 3355
1191 3355


In [ ]:
#print (G3.nodes)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
G4=nx.Graph()
# graph initialization for DIP data

In [ ]:
DIP_Proteins = set()

In [ ]:
with open(path+'/DIPData/dip.txt','r') as file:
  for i,line in enumerate(file):
          ln=line.split("\n")[0].split()
          G4.add_edge(ln[0],ln[1])

          # reading each protein        
          for p in ln:
              # displaying the protein          
              #print(p) 
              DIP_Proteins.add(p.strip())

print (i)
print (len(G4.nodes), len(G4.edges))

print(len(DIP_Proteins))

#notes: total interactions: 2890
#number of nodes: number of unique proteins: 2063

2889
2063 2890
2063


In [ ]:
textfile = open(path+"/DIPData/UniqueProteins.txt", "w")
for element in DIP_Proteins:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
edges_1st = set(G2.edges())
edges_2nd = set(G3.edges())
edges_DIP = set(G4.edges())

intersecting_edges1 = edges_1st & edges_DIP
intersecting_edges2 = edges_2nd & edges_DIP
print(len(intersecting_edges1))
print(len(intersecting_edges2))

#notes: The first 3355 data pairs in NIP-RW dataset are positive interaction as they intersect with DIP edges.
# The second 3355 data pairs in NIP-RW dataset are negative interactions as they do not have any intersection with DIP edges. 
# That's fine

1180
0


In [ ]:
ALPID=list(set(list(G2.nodes())+list(G3.nodes())+list(G4.nodes())))
print(len(ALPID))
f=open(path+'/FastaSeq/Protein.txt',"w")
f.write("\n".join(ALPID));f.close()

2882


In [ ]:
E2=edges_DIP

## Next Tasks
####   1) Make some random edges from the set of vertices of Graph G3, as per your code in G3, we have 1191 nodes are there, so we start with this nodes and assume that the edges are represented as E1 and consider E2 contain the set of positive edges from DIP.
####   2) Once the edges are generated find the set difference between E1 and E2 as E4 and we can compute this using following operations
####      E3 = E1 & E2   # intersecting edges which is nothing but the positive edges, so we need to remove these edges from our generated set of random regerated edge pairs to find the initial negative set
####      E4 = E1-E3     # subtract the intersecting set of edges(E3) from the E1 as E4 , this set (E4) represent our initial negative set





## One update is required in our previous task as step 0
## We need to create the negative pair between distantly located proteins,
## for this 1st of all we need to retrieve the location of each protein from 1191 proteins
## simple way to do this is to submit these proteins into the uniprot and only retrieve the Subcellular location from uniprot database



In [ ]:
def loadfile(fi):
  l=[]
  f=open(path+"/SubCC/{}".format(fi),"r")
  for line in f:
    l.append(line.split()[0])
  return list(set(l))

In [ ]:
#1 Nucleus_0191.txt 7982
#2 Mitochondrion_0173.txt 11454
#3 Vacuole_0272.txt 4
#4 Cyto_0086.txt 1261
#5 ERT_0095.txt 1586
#6 Golgi_Apar_0132.txt 205
#7 Peroxisome_0204.txt 27

In [ ]:
fl=[x for x in os.listdir(path+"/SubCC") if x.endswith(".txt")]
ssd={}
for i,fi in enumerate(fl):
 
  xi=loadfile(fi)
  for a in xi:
    ssd[a]=i+1
  print(i+1,fi,len(xi))
with open(path+"/SubCC/SubCCmap.json","w") as f:
  json.dump(ssd,f)

1 ERT_0095.txt 1586
2 Cyto_0086.txt 1261
3 Nucleus_0191.txt 7982
4 Mitochondrion_0173.txt 11454
5 Peroxisome_0204.txt 27
6 Vacuole_0272.txt 4
7 Golgi_Apar_0132.txt 205


In [ ]:
# Step 1:
# Generate all Pair of edges from the nodes of graph G3 (as per your earlier computation)
L=G3.nodes
G5=nx.Graph()
for i,a in enumerate(L):
  for j,b in enumerate(L):
    if i<j and a!=b:
      if a in ssd and b in ssd:
        if ssd[a]!=ssd[b]:
          G5.add_edge(a,b)
print(len(G5.edges))

#output: 52934

52934


In [ ]:
E1 = set(G5.edges()) #random edges generated from Graph G3 nodes
E2 = set(G4.edges()) #edges from DIP
#print(len(E1))
#print(len(E2))

E3=E1 & E2  # intersecting Edges // positive edges
E4=E1-E3    #Initial negative set
print(len(E4))

#edges reduced from 708645 to 708445 after removing intersecting edges from DIP
#E4: initial negative set 
#output: 52934

52934


In [ ]:
EE4=[]
for k in E4:
  sk=[k[0],k[1],0]
  EE4.append(sk)
print(len(EE4))
#output: 52934

52934


In [ ]:
52934/3355

15.777645305514158

In [ ]:
with open(path+"/traindata/ITR-1/ITR-1-Neg.json","w") as f:
  json.dump(list(EE4),f)

In [ ]:
with open(path+"/traindata/ITR-1/ITR-1-Neg.json","r") as f:
  EEX4=json.load(f)
for x in EEX4:
  print (x)
  break  

['P12235', 'Q8TEW0', 0]


In [ ]:
import random

In [ ]:
l1=[1,3,4,7,6,11,12,14,15,56]
S=random.sample(l1,4)
print(S)
S=random.sample(l1,4)
print(S)
S=random.sample(l1,4)
print(S)

[1, 15, 14, 6]
[56, 14, 4, 6]
[11, 56, 6, 15]


In [ ]:
#print(random.sample(E4, 3)) 
#Generate 3355*5 random protein pairs from E4 list of negative protein interaction edges
SampleNeg=random.sample(EE4, 3355*14)
#print (SampleNeg)

In [ ]:
with open(path+"/traindata/ITR-1/ITR-1-Neg_33550.json","w") as f:
  json.dump(SampleNeg,f)


In [ ]:
#Diving the sample negative set that is generated into two parts with 8387 protein pairs each
length = len(SampleNeg)
middle_index = length//2
first_half_negative = SampleNeg[:middle_index]
print(len(first_half_negative))
second_half_negative= SampleNeg[middle_index:]
print(len(second_half_negative))

23485
23485


In [ ]:
#positive interacting pairs list: 3355 pairs
print(type(list(G2.edges)))
SP=list(G2.edges)

PE4=[]
for k in SP:
  sk=[k[0],k[1],1]
  PE4.append(sk)
print(len(PE4))
SamplePos=PE4
#print(SamplePos)
#print (PE4)

with open(path+"/traindata/ITR-1/ITR-1-Pos_3355.json","w") as f:
  json.dump(SamplePos,f)


<class 'list'>
3355


In [ ]:
#Dividing the positive interacting protein pairs set (G2.edges) into two parts 
lengthpos=len(SamplePos)
middle_index=lengthpos//2
#print(middle_index)
#first_half_positve = SamplePos[:middle_index]
#second_half_positive= SamplePos[middle_index:]
first_half_positve = SamplePos
second_half_positive= SamplePos #[middle_index:]

In [ ]:
def JSDM(path,fn,D):
  with open(path+fn,"w") as f:
    json.dump(D,f)


In [ ]:
#forming two sets each with the ration of 1:10 positive to negative interacting protein pairs. SampleStart1 and SampleStart2
SampleStart1=[]
SampleStart1.extend(first_half_positve)
SampleStart1.extend(first_half_negative)

#random.shuffle(SampleStart1)
fn="/traindata/ITR-1/SetA.json"
JSDM(path,fn,SampleStart1)


In [ ]:
SampleStart2=[]
SampleStart2.extend(second_half_positive)
SampleStart2.extend(second_half_negative)
#random.shuffle(SampleStart2)
#print(SampleStart2)
print (len(SampleStart2))
fn="/traindata/ITR-1/SetB.json"
JSDM(path,fn,SampleStart2)

26840
